# Algebraic Models over $\mathbb{F}_{2^n}$

In [3]:
load("Poseidon2b.sage")

## Helpers

In [128]:
def gen_vector_from_dict(sol, F, t, name='x'):
    sol_dict = {str(var): val for var, val in sol.items()}
    vec = vector(F, [0] * t)
    for i in range(t):
        key = f'{name}{i}'
        if key in sol_dict:
            vec[i] = sol_dict[key]
    return vec

def solve_system(eqs, purpose='unknown', debug=False):
        F = eqs[0].base_ring()
        varnames = list({v for f in eqs for v in f.variables()})
        Q = PolynomialRing(F, names=varnames)
        if debug:
            print(f"{purpose} equations: {len(eqs)} equations in {len(varnames)} variables (degs {[f.degree() for f in eqs]})", flush=True)
        if len(eqs) == 1:
            f = Q(eqs[0]).monic().squarefree_part()
            sols = f.roots(multiplicities=False, algorithm='ntl')
            sol = [{Q.gen(): s} for s in sols]
        else:
            sol = Ideal([Q(f) for f in eqs]).variety()
        if len(sol) == 0:
            raise ValueError(f"No solution for the {purpose} equations found.")
        return sol

@parallel(ncpus=4)
def subvariety_task(polys, fixed_variable, fixed_value):
    """
    Fix one variable to a given value and solve the resulting system.

    Args:
    - polys (list): list of polynomials
    - fixed_variable (Sage variable): variable to be fixed
    - fixed_value (Finite field element): value to which variable is fixed
    """
    I = Ideal(polys + [fixed_variable - fixed_value])
    solutions = I.variety()
    return [dict(sol) for sol in solutions] # Otherwise we get INVALID DATA 'KeyConvertingDict' object has no attribute 'key_conversion_function'

def variety_branched(polys, fixed_index=0):
    R = polys[0].parent()
    if R.ngens() == 1:
        if len(polys) != 1:
            raise ValueError("Only one polynomial allowed in univariate case.")
        return solve_system(polys)

    fixed_variable = R.gens()[fixed_index]
    jobs = [(polys, fixed_variable, fixed_value) for fixed_value in R.base_ring()]

    solutions = []
    for args, res in subvariety_task(jobs):
        solutions.extend(res)

    return solutions

In [184]:
field_to_hex = lambda F, x: f"{x.to_integer() if F.degree() > 1 else ZZ(x):0{ceil(F.degree()/4)}x}"

def strpoly(poly):
    F = poly.base_ring()
    coeffs = [field_to_hex(F, c) for c in poly.coefficients()]
    monoms = [str(m) for m in poly.monomials()]
    return " + ".join(f"{c}*{m}" for c, m in zip(coeffs, monoms)) if poly != 0 else "0"

def strvec(v, polyvec=False):
    def elem_to_str(x):
        if polyvec:
            return strpoly(x)
        else:
            return field_to_hex(x.parent(), x)
    return "(" + ", ".join(elem_to_str(x) for x in v) + ")"


## Round Skipping Tricks

### Skipping the first full (external) round

In [ ]:
def skip_external_round(P, k1, k2, debug=False):
    """
    Find D-dimensional affine subspace to skip first external round of Poseidon2b instances.
    
    Args:
    - P (Object): Poseidon2b instance
    - k1 (int): Number of constrained inputs (Sponge: capacity for preimage attack, CICO-k: k)
    - k2 (int): Number of constrained outputs (Sponge: digest for preimage attack, CICO-k: k)

    Returns:
    - V, p: Affine subspace after first non-linear layer
    - r: full round number where subspace is placed (1 for first round)
    """
    if debug:
        print(f"Generate affine subspace to skip the first external round of {P.name} (k1={k1}, k2={k2})", flush=True)

    if k1 == 0:
        raise ValueError("Model not applicable to compression mode.")
    if P.rf < 1:
        raise ValueError("No initial full round to be skipped.")
    if P.t - k1 <= k2:
        raise ValueError("Not enough DOFs available to construct affine subspace.")

    k = max(k1, k2)
    if k > floor((P.t - 2*k) / k):
        raise ValueError("Not enough DOFs available to construct affine subspace: k > M_{k-1}.")

    # Introduce variables after first non-linear layer and go back to permutation input
    R = PolynomialRing(P.F, names=[f'x{i}' for i in range(P.t)])
    X = P.Minit_inv * (P.SubWordsRF(vector(R.gens()),inv=True) - P.rcons[0])

    if debug:
        print("System to solve:", flush=True)
        for eq in list(X)[-k:]:
            print(strpoly(eq), flush=True)

    # Split summands in equations modeling the constrained inputs into two parts, one for the coset and one for the basis

    # ------------------------------------------------------------------------------------------------------------
    # (I) Find the coset of the affine subspace
    # eqs_coset: a square system (k equations in k variables), used to derive the coset of the affine subspace

    if debug:
        print("--- Finding coset ---", flush=True)

    eqs_coset = [sum(f.terms()[-(k+1):]) for f in X[-k:]] 
    sol_coset = solve_system(eqs_coset, purpose='coset')
    coset = gen_vector_from_dict(sol_coset[0], P.F, P.t)

    if debug:
        for eq in eqs_coset:
            print(strpoly(eq), flush=True)
        print("Coset:")
        print(strvec(coset), flush=True)

    # ------------------------------------------------------------------------------------------------------------
    # (II) Find the basis of the affine subspace
    # eqs_basis: underdefined system of k equations in t−k variables (t-k >= k), used to derive the basis of the affine subspace
    # eqs_basis = [sum(f.terms()[:-(k+1)]) for f in X[-k:]] 
    
    if debug:
        print("--- Finding basis ---", flush=True)

    def fix_variables(R, eqs):
        # Fix variables until number of unknowns == number of equations
        tofix = R.ngens() - len(eqs)
        fixed = {}
        for v in R.gens()[::-1]: # fix variables starting from the end
            if tofix == 0:
                break
            fixed[v] = P.F.one()
            tofix -= 1
        return fixed

    basis = []
    eqs_basis = []
    # Split each equation in eqs_basis into subsums by grouping terms with the same index modulo k
    # Using x{j} ~ y_{j}^3 * x_{j mod k}, we get linear equations.
    idxs = [[i for i in range(P.t-k) if i % k == j] for j in range(k)]
    for i, idx in enumerate(idxs):
        M = P.Minit_inv[-k:, idx]
        R = PolynomialRing(P.F, names=[f'x{i}' for i in idx])
        eqs = list(M * vector(R.gens()))
        fixed = fix_variables(R, eqs) # fix variables such that a well-defined system remains
        eqs_basisvec = [f.specialization(fixed) for f in eqs]
        sol_basisvec = solve_system(eqs_basisvec, purpose='basis', )
        basisvec = gen_vector_from_dict({**fixed, **sol_basisvec[0]}, P.F, P.t)
        basis.append(P.SubWordsRF(basisvec))

        if debug:
            print(f"- System L{i}:", flush=True)
            for eq in eqs_basisvec:
                print(strpoly(eq), flush=True)
            print("Solution: ", {y : P.field_to_hex(val) for y, val in {**fixed, **sol_basisvec[0]}.items()}, flush=True)
    
    if debug:
        print("Basis vectors:")
        for basisvec in basis:
            print(strvec(basisvec), flush=True)

    return span(basis, P.F), coset, 1

#### Testing with Toy instance - Example from Appendix A.1.1

In [186]:
P8 = Poseidon2b(t=8, n=7, alpha=3, rF=4, rP=5, toy=True) # set toy=True to allow generation of toy versions of Poseidon2b
print(P8)

POSEIDON2B instance over GF(2^7), t=8, alpha=3; rF=4=2+2, rP=5


In [187]:
k1, k2 = 2, 1
V, p, r = skip_external_round(P=P8, k1=k1, k2=k2, debug=True)

R = PolynomialRing(P8.F, names=['y0', 'y1'])
v = vector(R, R.gens()) * V.basis_matrix() + p

x = P8.Minit_inv * (P8.SubWordsRF(v,inv=True) - P8.rcons[0]) # Calculating back to permutation input
assert(all([xi == 0 for xi in x[-k2:]]))

Generate affine subspace to skip the first external round of Poseidon2b (k1=2, k2=1)
System to solve:
5e*x0^85 + 3a*x1^85 + 64*x2^85 + 2f*x3^85 + 3f*x4^85 + 74*x5^85 + 4b*x6^85 + 5e*x7^85 + 2c*1
15*x0^85 + 71*x1^85 + 64*x2^85 + 64*x3^85 + 2a*x4^85 + 61*x5^85 + 4b*x6^85 + 4b*x7^85 + 41*1
--- Finding coset ---
4b*x6^85 + 5e*x7^85 + 2c*1
4b*x6^85 + 4b*x7^85 + 41*1
Coset:
(00, 00, 00, 00, 00, 00, 03, 4e)
--- Finding basis ---
- System L0:
5e*x0 + 64*x2 + 3f*1
15*x0 + 64*x2 + 2a*1
Solution:  {x4: '1', x2: '0', x0: '2'}
- System L1:
3a*x1 + 2f*x3 + 74*1
71*x1 + 64*x3 + 61*1
Solution:  {x5: '1', x1: '2', x3: '0'}
Basis vectors:
(08, 00, 00, 00, 01, 00, 00, 00)
(00, 08, 00, 00, 00, 01, 00, 00)


### Skipping several partial (internal) rounds

In [188]:
def skip_internal_rounds(P, k1, k2, debug=False, randcoset=False):
    """
    Find D-dimensional affine subspace to skip t-D internal rounds of Poseidon2b instances, where D = k1 + k2.
    Return source and target subspace
    
    Args:
    - P (Object): Poseidon2b instance
    - k1 (int): Number of constrained inputs (Sponge: capacity for preimage attack, Compression: 0 for preimage attack, CICO-k: k)
    - k2 (int): Number of constrained outputs (Sponge/Compression: digest for preimage attack, COCO-k: k)

    Returns:
    - V, p: Affine subspace after first non-linear layer
    - s: partial round number where subspace is placed (1 for first round)
    """
    l = min(P.rP, P.t-(k1+k2))
    if debug:
        print(f"Generate affine subspace to skip {l} internal rounds of {P.name} (k1={k1}, k2={k2})", flush=True)

    # Given M, return basis of source and target subspace
    # Number of rounds we want to skip, 1 <= l <= t-1, i.e., length of subspace trail
    if not (1 <= l <= P.t-1):
        raise ValueError(f"Invalid number of rounds to skip: {l} not in [{1},{P.t-1}]")
    if P.MP.multiplicative_order() < l:
        raise ValueError(f"MP order is {P.MP.multiplicative_order()} < l = {l}.")
    
    W = VectorSpace(P.F, P.t)
    
    # Generate affine subspace
    e0 = identity_matrix(P.F, P.t)[0]
    G = [e0 * P.MP**i for i in range(0,l)] # Generating set for subspace complement
    V = W.span(G).complement()
    if V.dimension() != P.t - l:
        raise ValueError(f"Invalid dimension for subspace: {V.dimension()} != {P.t - l}")
    # Target subspace: V*(P.MP^l).transpose()
    coset = V.complement().random_element() if randcoset else W.zero()
    
    # Position subspace in partial rounds (https://eprint.iacr.org/2025/954.pdf, p22)
    # Position subspace in the middle of the partial rounds
    s = floor((P.rP - l)/2)

    return V, coset, s

In [191]:
P8 = Poseidon2b(t=8, n=7, alpha=3, rF=4, rP=5, toy=True) # set toy=True to allow generation of toy versions of Poseidon2b
print(P8)

POSEIDON2B instance over GF(2^7), t=8, alpha=3; rF=4=2+2, rP=5


In [192]:
k1, k2 = 2, 2
V, gamma, s = skip_internal_rounds(P=P8, k1=2, k2=2)
l = P8.t - V.dimension()
print(V)
print('Length of subspace trail:', l)

Vector space of degree 8 and dimension 4 over Finite Field in X of size 2^7
Basis matrix:
[                              0                               1                               0                               0                               0               X^5 + X^4 + X + 1                 X^6 + X^4 + X^3             X^6 + X^5 + X^3 + X]
[                              0                               0                               1                               0                               0                         X^6 + X     X^6 + X^5 + X^4 + X^3 + X^2   X^5 + X^4 + X^3 + X^2 + X + 1]
[                              0                               0                               0                               1                               0         X^5 + X^3 + X^2 + X + 1                         X^2 + X                       X^5 + X^3]
[                              0                               0                               0                               0        

In [193]:
from sage.geometry.hyperplane_arrangement.affine_subspace import AffineSubspace

print("---- Start subspace ----")
A1 = AffineSubspace(V=V,p=p)
print(A1)

print("---- Target subspace ----")
# This is the subspace where we should get after r partial rounds, if we start in A1
q = P8(gamma, rstart=P8.rf, rend=P8.rf + l)
A2 = AffineSubspace(V=V*(P8.MP^l).transpose(), p=q)
print(A2)

print("---- Check if trail is valid ----")
# A1.random_element()
v_A1_lin = A1.linear_part().random_element()
v_A1 = v_A1_lin + gamma

# Check if applying l partial rounds to element of A1 yields element in A2
#w_A2 = InternalSubspaceRounds(P, v_A1, l, debug=False)
w_A2 = P8(v_A1, rstart=P8.rf, rend=P8.rf + l)
print(f"v + g in A1  =>  w = RI^l(v + g) in A2:  \t", w_A2 in A2)
print(f"v + g in A1  =>  w == MI^l * v + RI^l(g):\t", w_A2 == P8.MP^l * v_A1_lin + q)

# Same again, but for general representation of element and fixed gamma
R = PolynomialRing(P8.F, [f'x{i}' for i in range(V.dimension())])
v = vector(R.gens()) * V.basis_matrix() # Element in subspace Sr: x0*b0 + x1*b1 + ...
#w = InternalSubspaceRounds(P, v + gamma, l, debug=False)
w = P8(v + gamma, rstart=P8.rf, rend=P8.rf + l)
print(f"v + g in A1  =>  w == MI^l * v + RI^l(g):\t", w == P8.MP^l * v + q)

---- Start subspace ----
Affine space p + W where:
  p = (0, 0, 0, 0, 0, 0, X + 1, X^6 + X^3 + X^2 + X)
  W = Vector space of degree 8 and dimension 4 over Finite Field in X of size 2^7
Basis matrix:
[                              0                               1                               0                               0                               0               X^5 + X^4 + X + 1                 X^6 + X^4 + X^3             X^6 + X^5 + X^3 + X]
[                              0                               0                               1                               0                               0                         X^6 + X     X^6 + X^5 + X^4 + X^3 + X^2   X^5 + X^4 + X^3 + X^2 + X + 1]
[                              0                               0                               0                               1                               0         X^5 + X^3 + X^2 + X + 1                         X^2 + X                       X^5 + X^3]
[                         

## Algebraic Models

**Model Construction Interface**: `model_{model}_{dof}(P, k1, k2)`

This family of functions defines the algebraic representation of Poseidon2b under different modeling choices and degrees-of-freedom assumptions.  
Each function constructs a system of polynomial equations that capture the internal state transitions of the permutation, tailored to a specific modeling abstraction and strategy for obtaining a **square system** (same number of equations and variables).

**Function Signature**
```python
model_{model}_{dof}(P, k1, k2)
"""
    Construct the algebraic model for the Poseidon2b permutation in specific attack scenario
    under the specified modelling strategy and degrees-of-freedom assumption.

    Model type:
        {model} — either
            (1) Direct Forward Equations, or
            (2) Working at Round Level.

    Degrees of Freedom (DoF) strategy:
        {dof} — one of
            (a) 'skipnone'    : random input variable guessing,
            (b) 'skipfull'    : skipping the first full round,
            (c) 'skippartial' : skipping several partial rounds.

    Args:
        P (Hades): 
            The Poseidon2b permutation instance following the Hades design strategy.
        k1 (int): 
            Number of constrained inputs (Sponge: capacity for preimage attack, Compression: 0 for preimage attack, CICO-k: k)
        k2 (int): 
            Number of constrained outputs (Sponge/Compression: digest for preimage attack, COCO-k: k)

    Returns:
        tuple:
            A pair (E, V, p, r) where:
                E — list of polynomial equations defining the model.
                V — vectorspace of involved affine subspace V + p, in any, None otherwise.
                p — coset of involved affine subspace V + p, in any, None otherwise.
                r - placement of the involved affine subspace, if any, None otherwise.
    """
```

### (1) Direct Forward Equations

#### (a) Random guessing of input variables

In [218]:
@parallel(ncpus=4)
def model_fw_skipnone(P, k1, k2, debug=False, recover=True):
    """
    Forward model without round-skipping. Random guessing of variables for square system.

    Args:
    - P (Hades): Permutation following Hades design strategy.
    - k1 (int): Number of constrained inputs (Sponge: capacity for preimage attack, Compression: 0 for preimage attack, CICO-k: k)
    - k2 (int): Number of constrained outputs (Sponge/Compression: digest for preimage attack, COCO-k: k)

    Returns:
    - eqs (list): List of equations representing the model.
    - None, None, None: Dummy outputs for compatibility with other models.
    """
    if P.t - (k1 + k2) < 0:
        raise ValueError(f"Overdefined system.")

    # --------------------------------------------------------------
    # Variables: t-k1 input variables, set some to zero ("randomly set") such that we have k2 variables left
    # --------------------------------------------------------------
    R = PolynomialRing(P.F, names=[f'x{i}' for i in range(k2)])
    x = vector(R, list(R.gens()) + [0]*(P.t-k2))

    # --------------------------------------------------------------
    # Equations: k2 equations to map hash value
    # --------------------------------------------------------------
    result = P(x)
    if k1 == 0: # compression
        result += x
    eqs = list(result[-k2:])

    dExp = P.alpha**P.rF * P.alpha**P.rP
    if not all([f.degree() == dExp for f in eqs]):
        print("(x -> y)", [f.degree() for f in eqs], f"Expected: {dExp}", flush=True)
        assert(recover)
    if debug:
        print(f"{len(eqs):3d} eqs of degree {dExp:3d} = {P.alpha}^{P.rF} * {P.alpha}^{P.rP}", flush=True)
    
    nExp = k2
    if not len(eqs) == nExp:
        print(f"Warning: {len(eqs)} eqs found, but {nExp} expected", flush=True)
        assert(recover)
    if not R.ngens() == nExp:
        print(f"Warning: {R.ngens()} vars found, but {nExp} expected", flush=True)
        assert(recover)
    if debug:
        print(f"TOTAL: {len(eqs)} equations in {R.ngens()} variables\n", flush=True)

    return eqs, None, None, None

In [219]:
P = Poseidon2b(t=8, n=7, alpha=3, rF=4, rP=0, toy=True) # set toy=True to allow generation of toy versions of Poseidon2b
print(P, flush=True)

eqs, _, _, _ = model_fw_skipnone(P=P, k1=k1, k2=k2, debug=True, recover=True)
#sols = ideal(eqs).variety()
sols = variety_branched(eqs)
print("Solutions:", sols, flush=True)

for sol in sols:
    x = gen_vector_from_dict(sol, P.F, P.t)
    y = P(x)

    if k1 == 0:  # compression
        y += x
    print(f"x: {x}\ny: {y}", flush=True)
    assert(all([yi == 0 for yi in y[-k2:]]))

POSEIDON2B instance over GF(2^7), t=8, alpha=3; rF=4=2+2, rP=0


  1 eqs of degree  81 = 3^4 * 3^0
TOTAL: 1 equations in 1 variables

Solutions: [{x0: X^5 + X^4 + X^2 + X + 1}, {x0: X^6 + X^3}, {x0: X^6 + X^4 + X^2}]
x: (X^5 + X^4 + X^2 + X + 1, 0, 0, 0, 0, 0, 0, 0)
y: (X^5 + X^4 + X^2 + 1, X^5 + X^4 + 1, X^5 + X^4 + 1, X^5 + X^4 + X^3 + X^2 + X, X^5 + X^4 + X + 1, X^5 + X^4 + X^2 + 1, X^4 + X, 0)
x: (X^6 + X^3, 0, 0, 0, 0, 0, 0, 0)
y: (X^6 + X^4 + 1, X^5 + X^3 + 1, X^6 + X^5 + X^4 + X + 1, X^5 + X^4 + X^2 + X + 1, X^6 + X^3 + X + 1, X^5 + X^3 + X + 1, X^6 + X^4 + X^3, 0)
x: (X^6 + X^4 + X^2, 0, 0, 0, 0, 0, 0, 0)
y: (X^5 + X^2 + X, X^5 + X^4 + X^3 + X^2, X^6 + X^2 + X, X^6 + X^5 + X^4, X^2 + X, X^5 + X^4 + X^3 + X^2 + X, X^6 + X^5 + X^4 + X^3 + X + 1, 0)


#### (b) Skipping the first external full round

In [220]:
@parallel(ncpus=4)
def model_fw_skipfull(P, k1, k2, debug=False, recover=True):
    """
    Forward model skipping one full round via a d-dimensional affine subspace.
    Consuming all available DoF.

    Args:
    - P (Hades): Permutation following Hades design strategy.
    - k1 (int): Number of constrained inputs (Sponge: capacity for preimage attack,CICO-k: k)
    - k2 (int): Number of constrained outputs (Sponge/Compression: digest for preimage attack, COCO-k: k)

    Returns:
    - eqs: List of equations representing the model.
    - V, p: Affine subspace after first non-linear layer
    - r: full round number where subspace is placed (1 for first round)
    """
    if P.t - (k1 + k2) < 0:
        raise ValueError(f"Overdefined system.")
    if k1 == 0:
        raise ValueError("Model not applicable to compression mode.")
    if P.rf < 1:
        raise ValueError("No initial full round to be skipped.")

    # --------------------------------------------------------------
    # Variables: k2 variables, coefficients of the linear combination of basis vectors after first non-linear layer
    # --------------------------------------------------------------
    V, p, r = skip_external_round(P,k1,k2)
    V = V.subspace(V.basis_matrix()[:min(k1,k2),:]) # Sets some coefficients to zero if k2 < k1
    
    R = PolynomialRing(P.F, names=[f'x{i}' for i in range(V.dimension())])
    x = P.MF * (vector(R, list(R.gens())) * V.basis_matrix() + p)

    # --------------------------------------------------------------
    # Equations: k2 equations to map hash value
    # --------------------------------------------------------------
    result = P(x, rstart=1)
    if k1 == 0: # compression
        result += x
    eqs = list(result[-k2:])

    dExp = P.alpha**(P.rF-1) * P.alpha**P.rP
    if not all([f.degree() == dExp for f in eqs]):
        print("(x -> y)", [f.degree() for f in eqs], f"Expected: {dExp}", flush=True)
        assert(recover)
    if debug:
        print(f"{len(eqs):3d} eqs of degree {dExp:3d} = {P.alpha}^{P.rF-1} * {P.alpha}^{P.rP}", flush=True)
    
    nExp = k2
    if not len(eqs) == nExp:
        print(f"Warning: {len(eqs)} eqs found, but {nExp} expected", flush=True)
        assert(recover)
    if not R.ngens() == nExp:
        print(f"Warning: {R.ngens()} vars found, but {nExp} expected", flush=True)
        assert(recover)
    if debug:
        print(f"TOTAL: {len(eqs)} equations in {R.ngens()} variables\n", flush=True)

    return eqs, V, p, r

In [221]:
P = Poseidon2b(t=8, n=9, alpha=3, rF=4, rP=0, toy=True) # set toy=True to allow generation of toy versions of Poseidon2b
k1, k2 = 2, 1
#P.set_rounds(3, P.rP, rf=2)
print(P, flush=True)

eqs, V, p, r = model_fw_skipfull(P, k1=k1, k2=k2, debug=True, recover=True)
#sols = ideal(eqs).variety()
sols = variety_branched(eqs)

print("Solutions:", sols, flush=True)

for sol in sols:
    v = gen_vector_from_dict(sol, P.F, V.dimension()) * V.basis_matrix() + p
    x = P(P.MF * v, rend=r, inv=True)
    y = P(x)

    if k1 == 0:  # compression
        y += x
    print(f"x: {x}\ny: {y}", flush=True)
    assert(all([yi == 0 for yi in y[-k2:]]))

POSEIDON2B instance over GF(2^9), t=8, alpha=3; rF=4=2+2, rP=0
  1 eqs of degree  27 = 3^3 * 3^0
TOTAL: 1 equations in 1 variables

Solutions: [{x0: X^8 + X^7 + X^6 + X^5 + X + 1}]
x: (X^7 + X^6 + X^5 + X^4 + X^3 + 1, X^8 + X^7 + X^5 + X^4 + X^3 + X^2 + X + 1, X^8 + X^7 + X^6 + X^4 + X^3 + X^2 + X, X^8 + X^7 + X, X^6 + X^4 + X^2 + 1, X^8 + X^7 + X^6 + X^4 + X^3 + 1, 0, 0)
y: (X^8 + X^6 + X^5 + X^4 + X^3 + X^2 + X + 1, X^7, X^7 + X^5 + X^4 + 1, X^7 + X^6 + X^5 + 1, X^7 + X^6 + X^5 + X^4 + 1, X^6 + X^5 + X^2 + X, X^6 + X^4 + X^2, 0)


#### (c) Skipping several internal partial rounds

In [ ]:
@parallel(ncpus=4)
def model_fw_skippartial(P, k1, k2, debug=False, recover=True):
    """
    Forward model skipping t-(c+d) partial rounds via a (c+d)-dimensional affine subspace.
    Consuming all available DoF.

    Args:
    - P (Hades): Permutation following Hades design strategy.
    - k1 (int): Number of constrained inputs (Sponge: capacity for preimage attack, Compression: 0 for preimage attack, CICO-k: k)
    - k2 (int): Number of constrained outputs (Sponge/Compression: digest for preimage attack, COCO-k: k)

    Returns:
    - eqs: List of equations representing the model.
    - V, p: Affine subspace after first non-linear layer
    - s: partial round number where subspace is placed (default to middle of partial rounds)
    """
    if P.t - (k1 + k2) < 0:
        raise ValueError(f"Overdefined system.")
    if P.rP < P.t - (k1 + k2):
        raise ValueError(f"Invalid parameters: rP = {P.rP} < t-(k1+k2) = {P.t-(k1+k2)}")

    # --------------------------------------------------------------
    # Variables: k2 variables, coefficients of the linear combination of basis vectors after first non-linear layer
    # --------------------------------------------------------------
    V, p, s = skip_internal_rounds(P,k1,k2) 

    vars_in_x = [f'x{i}' for i in range(P.t-k1)]
    vars_V_coeff = [f'a{i}' for i in range(V.dimension())]    # coefficients for element of subspace V (for v)
    R = PolynomialRing(P.F, vars_in_x + vars_V_coeff)

     # Input to permutation
    x = vector(R, vars_in_x + [0]*k1) # k1 input elements fixed to zero

    # Element in linear part V of affine subspace V+p
    v = vector(R, vars_V_coeff) * V.basis_matrix()

    # --------------------------------------------------------------
    # Equations: 
    # - t equations to map input into affine subspace
    # - k2 equations to map hash value
    # --------------------------------------------------------------

    # Step 1: x -> v + p
    # t equations to map input into affine subspace
    result = P(x, rend=P.rf+s) - (v + p)
    eqs1 = list(result)
    
    dExp = P.alpha**P.rf * P.alpha**s
    if not all([f.degree() == dExp for f in eqs1]):
        print("(x -> v + p)", [f.degree() for f in eqs1], f"Expected: {dExp}", flush=True)
        assert(recover)
    if debug:
        print("(x -> v + p)", f"{len(eqs1):3d} eqs of degree {dExp:3d} = {P.alpha}^{P.rf} * {P.alpha}^{s}", flush=True)

    # Step 2: v -> w, p -> q
    # Element from one affine subspace maps to element from other affine subspace via linear transformation
    l = P.t - V.dimension()  # Length of subspace trail
    w = P.MP**l * v
    q = P(p, rstart=P.rf+s, rend=P.rf+s+l)

    # Step3: w + q -> y
    # k2 equations to map hash value
    result = P(w + q, rstart=P.rf+s+l)
    if k1 == 0: # compression
        result += x
    eqs3 = list(result[-k2:])

    dExp = P.alpha**(P.rP-(s+l)) * P.alpha**P.rf_
    if not all([f.degree() == dExp for f in eqs3]):
        print("(w + s -> y)", [f.degree() for f in eqs3], f"Expected: {dExp}", flush=True)
        assert(recover)
    if debug:
        print("(w + s -> y)", f"{len(eqs3):3d} eqs of degree {dExp:3d} = {P.alpha}^({P.rP}-({s}+{l})) * {P.alpha}^{P.rf_}", flush=True)

    # Summary
    eqs = eqs1 + eqs3
    nExp = P.t + k2
    if not len(eqs) == nExp:
        print(f"Warning: {len(eqs)} eqs found, but {nExp} expected", flush=True)
        assert(recover)
    if not R.ngens() == nExp:
        print(f"Warning: {R.ngens()} vars found, but {nExp} expected", flush=True)
        assert(recover)
    if debug:
        print(f"TOTAL: {len(eqs)} equations in {R.ngens()} variables\n", flush=True)

    return eqs, V, p, s

In [225]:
P = Poseidon2b(t=4, n=7, alpha=3, rF=2, rP=2, toy=True) # set toy=True to allow generation of toy versions of Poseidon2b
k1, k2 = 1, 1
print(P)

eqs, V, p, s = model_fw_skippartial(P, k1=k1, k2=k2, debug=True, recover=True)
#sols = ideal(eqs).variety()
sols = variety_branched(eqs)

print("Solutions:", sols, flush=True)

for sol in sols:
    x = gen_vector_from_dict(sol, P.F, P.t, name='x')
    y = P(x)

    # Additional correctness checks
    v = gen_vector_from_dict(sol, P.F, V.dimension(), name='a') * V.basis_matrix()
    l = P.t - V.dimension()
    w = P.MP**l * v
    q = P(p, rstart=P.rf+s, rend=P.rf+s+l)
    assert(P(x, rend=P.rf+s) == v + p) # x -> v + p
    assert(P(v + p, rstart=P.rf + s, rend=P.rf + s + l) == w + q) # v + p -> w + q
    assert(P(x, rend=P.rf + s + l) == w + q)

    if k1 == 0:  # compression
        y += x
    print(f"x: {x}\ny: {y}", flush=True)
    assert(all([yi == 0 for yi in y[-k2:]]))

POSEIDON2B instance over GF(2^7), t=4, alpha=3; rF=2=1+1, rP=2
(x -> v + p)   4 eqs of degree   3 = 3^1 * 3^0
(w + s -> y)   1 eqs of degree   3 = 3^(2-(0+2)) * 3^1
TOTAL: 5 equations in 5 variables



Solutions: [{a1: X^5 + X^2, a0: X^4 + X^2 + X, x2: X^6 + X^5 + X^4 + X^2, x1: X^6 + X^2 + X + 1, x0: X^4 + X^3 + X + 1}, {a1: X^5 + X^4 + X^3 + X^2 + X, a0: X^5 + X^4 + X^3 + X + 1, x2: X^4 + X^2 + X + 1, x1: X^6 + X^5 + X^4 + X^2 + X + 1, x0: X^5 + X^3 + X + 1}]
x: (X^4 + X^3 + X + 1, X^6 + X^2 + X + 1, X^6 + X^5 + X^4 + X^2, 0)
y: (X^5 + X^4 + X^3 + X^2 + 1, X^6 + X^5 + X^4 + X^3 + X^2 + X + 1, X^6 + X + 1, 0)
x: (X^5 + X^3 + X + 1, X^6 + X^5 + X^4 + X^2 + X + 1, X^4 + X^2 + X + 1, 0)
y: (X^5 + 1, X^3 + X + 1, X^5 + X^4 + X^3 + X + 1, 0)


### (2) Working at Round Level

#### (a) Random guessing of input variables

In [213]:
@parallel(ncpus=4)
def model_rd_skipnone(P, k1, k2, debug=False, recover=True):
    """
    Round-level model without round-skipping. Random guessing of variables for square system.

    Args:
    - P (Hades): Permutation following Hades design strategy.
    - k1 (int): Number of constrained inputs (Sponge: capacity for preimage attack, Compression: 0 for preimage attack, CICO-k: k)
    - k2 (int): Number of constrained outputs (Sponge/Compression: digest for preimage attack, COCO-k: k)

    Returns:
    - eqs (list): List of equations representing the model.
    - None, None, None: Dummy outputs for compatibility with other models.
    """
    if P.t - (k1 + k2) < 0:
        raise ValueError(f"Overdefined system.")
    if P.rf < 1 or P.rf_ < 1:
        raise ValueError(f"Invalid parameters: rf = {P.rf}, rf_ = {P.rf_}")

    # --------------------------------------------------------------
    # Variables: 
    # - t-k1 input variables, some set to zero ("randomly set") such that we have k2 variables left
    # - t*(RF-1) + RP round variables (one variable per S-box input, except first round since S-Box input linear in input variables)
    # --------------------------------------------------------------
    vars_in = [f'x{i}' for i in range(k2)]
    vars_rd = [[]] + [[f'r{r}_{i}' for i in range(P.t if P.is_full_round(r) else 1)] for r in range(1,P.R)]  
    R = PolynomialRing(P.F, names= vars_in + flatten(vars_rd))
    x = vector(R, vars_in + [0]*(P.t-k2))

    # --------------------------------------------------------------
    # Equations: 
    # - t equations per full round, 1 equation per partial round (except last round)
    # - k2 equations to map hash value
    # -----------------------------------------------------------------------
    eqs = []
    sout_this = P.SubWordsRF(P.Minit * x + P.rcons[0]) # first round S-box output
    for r in range(1, P.R):
        sout_prev = sout_this
        sin_this = (P.MF * sout_prev if P.is_full_round(r - 1) else P.MP * sout_prev) + P.rcons[r]
        rd_eqs = [(sin_this[i] - R(si)) for i, si in enumerate(vars_rd[r])] # link to round variables
        eqs += rd_eqs
        if debug:
            print(f"Link S-Box input of this round {r} ({'full' if P.is_full_round(r) else 'partial'}) to S-Box output of previous round ({'full' if P.is_full_round(r-1) else 'partial'}): {len(rd_eqs)} eqs")
        sout_this = P.SubWordsRF(vector(R, vars_rd[r])) if P.is_full_round(r) else P.SubWordsRP(vector(R, [R(vars_rd[r][0])] + list(sin_this[1:])))

    result = P.MF * sout_this
    if k1 == 0: # compression
        result += x
    eqs += list(result[-k2:])
    if debug:
        print(f"Link round {r} ({'full' if P.is_full_round(r) else 'partial'}) to output: {k2} eqs")

    dExp = P.alpha
    if not all([f.degree() == dExp for f in eqs]):
        print(f"Warning: Degree {[f.degree() for f in eqs if f.degree() != dExp]} eqs found, {dExp} expected", flush=True)
        assert(recover)
    if debug:
        print(f"{len(eqs):3d} = {P.t} * ({P.rF} - 1) + {P.rP} + {k2} eqs of degree {dExp:3d}", flush=True)

    nExp = P.t * (P.rF - 1) + P.rP + k2
    if not len(eqs) == nExp:
        print(f"Warning: {len(eqs)} eqs found, but {nExp} expected", flush=True)
        assert(recover)
    if not R.ngens() == nExp:
        print(f"Warning: {R.ngens()} vars found, but {nExp} expected", flush=True)
        assert(recover)
    if debug:
        print(f"TOTAL: {len(eqs)} equations in {R.ngens()} variables\n", flush=True)

    return eqs, None, None, None

In [214]:
P = Poseidon2b(t=4, n=7, alpha=3, rF=4, rP=1, toy=True) # set toy=True to allow generation of toy versions of Poseidon2b
k1, k2 = 1, 1
print(P, flush=True)

eqs, _, _, _ = model_rd_skipnone(P, k1=k1, k2=k2, debug=True, recover=True)
#sols = ideal(eqs).variety()
sols = variety_branched(eqs)
print("Solutions:", sols)

for sol in sols:
    x = gen_vector_from_dict(sol, P.F, P.t, name='x')
    y = P(x)

    if k1 == 0:  # compression
        y += x
    print(f"x: {x}\ny: {y}", flush=True)
    assert(all([yi == 0 for yi in y[-k2:]]))

POSEIDON2B instance over GF(2^7), t=4, alpha=3; rF=4=2+2, rP=1
Link S-Box input of this round 1 (full) to S-Box output of previous round (full): 4 eqs
Link S-Box input of this round 2 (partial) to S-Box output of previous round (full): 1 eqs
Link S-Box input of this round 3 (full) to S-Box output of previous round (partial): 4 eqs
Link S-Box input of this round 4 (full) to S-Box output of previous round (full): 4 eqs
Link round 4 (full) to output: 1 eqs
 14 = 4 * (4 - 1) + 1 + 1 eqs of degree   3
TOTAL: 14 equations in 14 variables

Solutions: [{r4_3: X^4, r4_2: X^6 + X^5 + X^4 + X^3 + X^2 + X, r4_1: 0, r4_0: X^6 + X^3 + 1, r3_3: 1, r3_2: X^5 + X^4 + X^3 + X + 1, r3_1: X^6 + X^5 + X^4, r3_0: X^5 + X, r2_0: X^3 + X^2, r1_3: X^6 + X^2 + 1, r1_2: X^6 + X^5 + X^4 + X^3 + X^2 + X, r1_1: X^3 + X^2 + 1, r1_0: X^6 + X^2 + X, x0: X^3 + X}, {r4_3: X^6 + X^4 + X^3 + X^2, r4_2: X^5 + X^4 + X^2 + 1, r4_1: X^5 + X^4 + X + 1, r4_0: X^6 + X^4 + X^3 + 1, r3_3: X^6 + X^4 + 1, r3_2: X^6 + X^4 + X^2, r3_1

#### (b) Skipping the first external full round

In [ ]:
@parallel(ncpus=4)
def model_rd_skipfull(P, k1, k2, debug=False, recover=True):
    """
    Round-level model skipping one full round via a k2-dimensional affine subspace.
    Consuming all available DoF.
    
    Args:
    - P (Hades): Permutation following Hades design strategy.
    - k1 (int): Number of constrained inputs (Sponge: capacity for preimage attack,CICO-k: k)
    - k2 (int): Number of constrained outputs (Sponge/Compression: digest for preimage attack, COCO-k: k)

    Returns:
    - eqs: List of equations representing the model.
    - V, p: Affine subspace after first non-linear layer
    - r: full round number where subspace is placed (1 for first round)
    """
    if P.t - (k1 + k2) < 0:
        raise ValueError(f"Overdefined system.")
    if P.rf < 1 or P.rf_ < 1:
        raise ValueError(f"Invalid parameters: rf = {P.rf}, rf_ = {P.rf_}")

    # --------------------------------------------------------------
    # Variables: 
    # - k2 variables for the coefficient vector after first non-linear layer
    # - t*(RF-2) + RP round variables (one variable per S-box input, except first round since S-Box input linear in input variables)
    # --------------------------------------------------------------
    if debug:
        print("Computing affine subspace to skip first full round...", flush=True)
    V, p, r = skip_external_round(P,k1,k2) 
    V = V.subspace(V.basis_matrix()[:min(k1,k2),:]) # Sets some coefficients to zero if k2 < k1

    vars_V_coeff = [f'x{i}' for i in range(V.dimension())]
    vars_rd = [[], []] + [[f'r{r}_{i}' for i in range(P.t if P.is_full_round(r) else 1)] for r in range(2,P.R)] 
     
    R = PolynomialRing(P.F, names= vars_V_coeff + flatten(vars_rd))
    x = vector(R, vars_V_coeff) * V.basis_matrix() + p # state after first nonlinear layer

    # --------------------------------------------------------------
    # Equations: 
    # - t equations per full round, 1 equation per partial round (except last round)
    # - k2 equations to map hash value
    # --------------------------------------------------------------
    eqs = []
    sout_this = P.SubWordsRF(P.MF * x + P.rcons[1]) if P.rf > 1 else P.SubWordsRP(P.MF * x + P.rcons[1]) # second round S-box output
    for r in range(2, P.R):
        sout_prev = sout_this
        sin_this = (P.MF * sout_prev if P.is_full_round(r - 1) else P.MP * sout_prev) + P.rcons[r]
        rd_eqs = [(sin_this[i] - R(si)) for i, si in enumerate(vars_rd[r])] # link to round variables
        eqs += rd_eqs
        if debug:
            print(f"Link S-Box input of this round {r} ({'full' if P.is_full_round(r) else 'partial'}) to S-Box output of previous round ({'full' if P.is_full_round(r-1) else 'partial'}): {len(rd_eqs)} eqs")
        sout_this = P.SubWordsRF(vector(R, vars_rd[r])) if P.is_full_round(r) else P.SubWordsRP(vector(R, [R(vars_rd[r][0])] + list(sin_this[1:])))

    result = P.MF * sout_this
    if k1 == 0: # compression
        result += x
    eqs += list(result[-k2:])
    if debug:
        print(f"Link round {r} ({'full' if P.is_full_round(r) else 'partial'}) to output: {k2} eqs")

    dExp = P.alpha
    if not all([f.degree() == dExp for f in eqs]):
        print(f"Warning: Degree {[f.degree() for f in eqs if f.degree() != dExp]} eqs found, {dExp} expected", flush=True)
        assert(recover)
    if debug:
        if P.rf > 1:
            print(f"{len(eqs):3d} = {P.t} * ({P.rF} - 2) + {P.rP} + {k2} eqs of degree {dExp:3d}", flush=True)
        else:
            print(f"{len(eqs):3d} = {P.t} * ({P.rF} - 1) + {P.rP} - 1 + {k2} eqs of degree {dExp:3d}", flush=True)

    nExp = P.t * (P.rF - 2) + P.rP + k2 if P.rf > 1 else P.t * (P.rF - 1) + P.rP - 1 + k2
    if not len(eqs) == nExp:
        print(f"Warning: {len(eqs)} eqs found, but {nExp} expected", flush=True)
        assert(recover)
    if not R.ngens() == nExp:
        print(f"Warning: {R.ngens()} vars found, but {nExp} expected", flush=True)
        assert(recover)
    if debug:
        print(f"TOTAL: {len(eqs)} equations in {R.ngens()} variables\n", flush=True)

    return eqs, V, p, r

In [227]:
P = Poseidon2b(t=8, n=9, alpha=3, rF=2, rP=4, toy=True) # set toy=True to allow generation of toy versions of Poseidon2b
k1, k2 = 2, 1
#P.set_rounds(3, P.rP, rf=2)
print(P, flush=True)

eqs, V, p, r = model_rd_skipfull(P, k1=k1, k2=k2, debug=True, recover=True)
#sols = ideal(eqs).variety()
sols = variety_branched(eqs)

print("Solutions:", sols, flush=True)

for sol in sols:
    v = gen_vector_from_dict(sol, P.F, V.dimension(), name='x') * V.basis_matrix() + p
    x = P.Minit_inv * (P.SubWordsRF(v, inv=True) - P.rcons[0])
    y = P(x)

    if k1 == 0:  # compression
        y += x
    print(f"x: {x}\ny: {y}", flush=True)
    assert(all([yi == 0 for yi in y[-k2:]]))

POSEIDON2B instance over GF(2^9), t=8, alpha=3; rF=2=1+1, rP=4
Computing affine subspace to skip first full round...
Link S-Box input of this round 2 (partial) to S-Box output of previous round (partial): 1 eqs
Link S-Box input of this round 3 (partial) to S-Box output of previous round (partial): 1 eqs
Link S-Box input of this round 4 (partial) to S-Box output of previous round (partial): 1 eqs
Link S-Box input of this round 5 (full) to S-Box output of previous round (partial): 8 eqs
Link round 5 (full) to output: 1 eqs
 12 = 8 * (2 - 1) + 4 - 1 + 1 eqs of degree   3
TOTAL: 12 equations in 12 variables

Solutions: [{r5_7: X^8 + X^6 + X^5 + X^3, r5_6: X^8 + X^7 + X^6 + X^5 + X^3, r5_5: X^7 + X^6 + X^4 + X^3 + X^2 + 1, r5_4: X^8 + X^6 + X^5 + X^4 + X^2 + X + 1, r5_3: X^8 + X^7 + X^4 + X^3, r5_2: X^7 + X^4 + X^3 + X^2 + 1, r5_1: X^8 + X^7 + X^5 + X^4 + X^3 + X, r5_0: X^8 + X^7 + X^6 + X^3 + X^2 + 1, r4_0: X^2, r3_0: X^5 + X^4, r2_0: X^7 + X^6 + X^4 + X^3 + X, x0: X^8 + X^7 + X^3 + X^2 + 

#### (c) Skipping several internal partial rounds

In [229]:
@parallel(ncpus=4)
def model_rd_skippartial(P, k1, k2, debug=False, recover=True):
    """
    Round-level model skipping t-(k1+k2) partial rounds via a (k1+k2)-dimensional affine subspace.
    Consuming all available DoF.

    Args:
    - P (Hades): Permutation following Hades design strategy.
    - k1 (int): Number of constrained inputs (Sponge: capacity for preimage attack, Compression: 0 for preimage attack, CICO-k: k)
    - k2 (int): Number of constrained outputs (Sponge/Compression: digest for preimage attack, COCO-k: k)

    Returns:
    - eqs: List of equations representing the model.
    - V, p: Affine subspace after first non-linear layer
    - s: partial round number where subspace is placed (default to start of partial rounds)    
    """
    if P.t - (k1 + k2) < 0:
        raise ValueError(f"Overdefined system.")
    if P.rf < 1:
        raise ValueError(f"Invalid parameters: rf = {P.rf}")
    
    l = P.t - (k1 + k2)  # Length of subspace trail = number of partial rounds that can be skipped
    if P.rP <= l:  # For simplicity, at least one partial round is performed after skipped ones
        raise ValueError(f"Invalid parameters: rP = {P.rP} < t-(k1+k2) = {P.t-(k1+k2)}")
    
    # --------------------------------------------------------------
    # Variables: 
    # - t-k1 input variables
    # - k1+k2 variables for the coefficient vector to map into the subspace
    # - t*(RF-1) + RP - l round variables (one variable per S-box input, except first round since S-Box input linear in input variables)
    # --------------------------------------------------------------
    V, p, s = skip_internal_rounds(P,k1,k2)
    s = 0 # for round-model, put skipped partial block at beginning
    assert(l == P.t - V.dimension()) # Length of subspace trail = number of partial rounds that can be skipped

    vars_in = [f'x{i}' for i in range(P.t-k1)]   # outer part variables (rate r = statesize t - capacity c elements)
    vars_V_coeff = [f'a{i}' for i in range(V.dimension())]    # coefficients for element of subspace V (for v)
    vars_rd = [[]] + [[f'r{r}_{i}' for i in range(P.t)] for r in range(1,P.rf)] + \
              [[]]*(l+1) + [[f'r{r}_{0}'] for r in range(P.rf + l + 1, P.rf + P.rP)] + \
              [[f'r{r}_{i}' for i in range(P.t)] for r in range(P.rf + P.rP, P.rF + P.rP)] # round variables
    
    R = PolynomialRing(P.F, vars_in + vars_V_coeff + flatten(vars_rd))

     # Input to permutation
    x = vector(R, vars_in + [0]*k1) # k1 input elements fixed to zero

    # Element in linear part V of affine subspace V+p
    v = vector(R, vars_V_coeff) * V.basis_matrix()

    # --------------------------------------------------------------
    # Equations: 
    # - t equations per full round (except first)
    # - t equations to map last full round into affine subspace
    # - 1 equation per partial round (except skipped ones and first after skipped one)
    # - k2 equations to map hash value
    # --------------------------------------------------------------
    eqs = []
    sout_this = P.SubWordsRF(P.Minit * x + P.rcons[0]) # first round S-box output
    for r in range(1, P.rf):
        sout_prev = sout_this
        sin_this = P.MF * sout_prev + P.rcons[r]
        rd_eqs = [(sin_this[i] - R(si)) for i, si in enumerate(vars_rd[r])] # link to round variables
        eqs += rd_eqs
        if debug:
            print(f"Link S-Box input of this round {r} ({'full' if P.is_full_round(r) else 'partial'}) to S-Box output of previous round ({'full' if P.is_full_round(r-1) else 'partial'}): {len(rd_eqs)} eqs")
        sout_this = P.SubWordsRF(vector(R, vars_rd[r]))
    
    # Map into affine subspace
    rd_eqs = P.MF * sout_this - (v + p)
    if debug:
        print(f"Link S-Box output of previous round {P.rf-1} ({'full' if P.is_full_round(P.rf-1) else 'partial'}) to affine subspace: {len(rd_eqs)} eqs")
    eqs += rd_eqs
    
    # Skip rounds
    w = P.MP**l * v
    q = P(p, rstart=P.rf, rend=P.rf+l)
    if debug:
        print(f"Skip {l} partial rounds: {P.rf} - {P.rf + l - 1}", flush=True)

    # Finish remaining rounds
    sout_this = P.SubWordsRP(w + q + P.rcons[P.rf + l])
    for r in range(P.rf + l + 1, P.R):
        sout_prev = sout_this
        sin_this = (P.MF * sout_prev if P.is_full_round(r - 1) else P.MP * sout_prev) + P.rcons[r]
        rd_eqs = [(sin_this[i] - R(si)) for i, si in enumerate(vars_rd[r])] # link to round variables
        eqs += rd_eqs
        if debug:
            print(f"Link S-Box input of this round {r} ({'full' if P.is_full_round(r) else 'partial'}) to S-Box output of previous round ({'full' if P.is_full_round(r-1) else 'partial'}): {len(rd_eqs)} eqs")
        sout_this = P.SubWordsRF(vector(R, vars_rd[r])) if P.is_full_round(r) else P.SubWordsRP(vector(R, [R(vars_rd[r][0])] + list(sin_this[1:])))
    
    result = P.MF * sout_this
    if k1 == 0: # compression
        result += x
    eqs += list(result[-k2:])
    if debug:
        print(f"Link round {P.R} ({'full' if P.is_full_round(P.R) else 'partial'}) to output: {k2} eqs")
    
    dExp = P.alpha
    if not all([f.degree() == dExp for f in eqs]):
        print(f"Warning: Degree {[f.degree() for f in eqs if f.degree() != dExp]} eqs found, {dExp} expected", flush=True)
        assert(recover)
    if debug:
        print(f"{len(eqs):3d} = {P.t} * ({P.rF} - 1) + {P.t} + {P.rP} - {l} - 1 + {k2} eqs of degree {dExp:3d}", flush=True)

    nExp = P.t*(P.rF - 1) + P.t + (P.rP - l - 1) + k2
    if not len(eqs) == nExp:
        print(f"Warning: {len(eqs)} eqs found, but {nExp} expected", flush=True)
    if not nExp == R.ngens():
        print(f"Warning: {R.ngens()} variables found, but {nExp} expected", flush=True)
    if debug:
        print(f"TOTAL: {len(eqs)} equations in {R.ngens()} variables\n", flush=True)
    
    return eqs, V, p, s

In [230]:
P = Poseidon2b(t=4, n=7, alpha=3, rF=2, rP=3, toy=True) # set toy=True to allow generation of toy versions of Poseidon2b
k1, k2 = 1, 1
print(P)

eqs, V, p, s = model_rd_skippartial(P, k1=k1, k2=k2, debug=True, recover=True)
#sols = ideal(eqs).variety()
sols = variety_branched(eqs)

print("Solutions:", sols, flush=True)
#assert(False)
for sol in sols:
    x = gen_vector_from_dict(sol, P.F, P.t, name='x')
    y = P(x)

    # Additional correctness checks
    v = gen_vector_from_dict(sol, P.F, V.dimension(), name='a') * V.basis_matrix()
    l = P.t - V.dimension()
    w = P.MP**l * v
    q = P(p, rstart=P.rf+s, rend=P.rf+s+l)
    assert(P(x, rend=P.rf+s) == v + p) # x -> v + p
    assert(P(v + p, rstart=P.rf + s, rend=P.rf + s + l) == w + q) # v + p -> w + q
    assert(P(x, rend=P.rf + s + l) == w + q)

    if k1 == 0:  # compression 
        y += x
    print(f"x: {x}\ny: {y}", flush=True)
    assert(all([yi == 0 for yi in y[-k2:]]))

POSEIDON2B instance over GF(2^7), t=4, alpha=3; rF=2=1+1, rP=3
Link S-Box output of previous round 0 (full) to affine subspace: 4 eqs
Skip 2 partial rounds: 1 - 2
Link S-Box input of this round 4 (full) to S-Box output of previous round (partial): 4 eqs
Link round 5 (full) to output: 1 eqs
  9 = 4 * (2 - 1) + 4 + 3 - 2 - 1 + 1 eqs of degree   3
TOTAL: 9 equations in 9 variables

Solutions: [{r4_3: X^6 + X^3 + X, r4_2: X^6 + X + 1, r4_1: X^6 + X^5 + X^4 + X^3 + 1, r4_0: X^6 + X^5 + X^4 + X^3 + X^2 + 1, a1: X^5 + X^4 + X^3 + X^2 + 1, a0: X^5 + X^4 + X^3, x2: X^6 + X^5 + X^4 + X + 1, x1: X^5 + X^2, x0: X^6 + X^5 + X^4 + X^3}, {r4_3: X^5 + X^4 + X^2 + 1, r4_2: X^5, r4_1: X^4 + X, r4_0: X^6 + X^5 + X^4, a1: X^5 + X^3 + X^2, a0: X^6 + X^2 + 1, x2: X^6 + 1, x1: X^6 + X^2 + 1, x0: X^4 + X^3 + 1}]
x: (X^6 + X^5 + X^4 + X^3, X^5 + X^2, X^6 + X^5 + X^4 + X + 1, 0)
y: (X^6 + X, X^6 + X^3 + X^2 + X + 1, X^5 + X^4 + X^2, 0)
x: (X^4 + X^3 + 1, X^6 + X^2 + 1, X^6 + 1, 0)
y: (X^3 + 1, X^5 + X^4 + X^2 +